<h1>Final Project Notebook</h1>

<h2>Importing/Installing Relevant Libraries</h2>

In [1]:
!pip install bs4
!pip install geopy
!pip install folium
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import geopy as gepoy
from geopy.geocoders import Nominatim
import folium as folium
from sklearn.cluster import KMeans
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize


/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=cde7220647699261f2bee48abc3115347188bc91afae7ccf5295f5f887a917b0
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/

<h3>Setting up Beautiful Soup</h3>

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(url).text
soup = BeautifulSoup(data, 'html5lib')

<h3>Scraping Table</h3>

In [3]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


<h3>Adding Longitude and Latitude to DataFrame</h3>

In [4]:
url2 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
df2 = pd.read_csv(url2)
df = pd.merge(df, df2, on='Postal Code', how='inner')
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


<h3>Performing KMeans Clustering on Data</h3>

In [5]:
kclusters = 5

coords = df[["Latitude", "Longitude"]]

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(coords)
df.insert(0, 'Cluster Labels', kmeans.labels_)

<h3>Visualizing Clusters</h3>

In [6]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print((latitude, longitude))

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['Latitude'], df['Longitude'], df['Neighborhood'], df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

(43.6534817, -79.3839347)


<h3>Getting Venue Information from FourSquare</h3>

In [8]:
CLIENT_ID = 'N3E5BJBASSRHGZPLTROGRXCKCKAHGH034JN4K4U3A5DOXEOW' # your Foursquare ID
CLIENT_SECRET = 'GWYLLB14V2R12OPVAF3V1VMXUEAUNHGYC0KO23E3SVXLUFKW' # your Foursquare Secret
ACCESS_TOKEN = '1NLIGRDKZGJZBBMUQFAA15CADMRFAXTKPLBELZUPA52Q4NF5' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30

venues_df = pd.DataFrame()
radius = 322 #approximately 2 miles
index_1=0
index_2=0
i=0
while i < df.shape[0]:
    latitude = df["Latitude"][i]
    longitude = df["Longitude"][i]
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, radius, LIMIT)
    results = requests.get(url).json()
    while(index_2<len(results['response']['groups'][0]['items'])):
        temp_df = json_normalize(results['response']['groups'][0]['items'][index_2]['venue'])

        venues_df = venues_df.append(temp_df)
        index_2+=1
    index_2=0
    index_1+=1
    venues_df.drop_duplicates(subset ="id",keep = False, inplace = True)
venues_df

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/ipykernel/__main__.py:19: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


KeyError: 'groups'

<h3>Cleaning Venues DataFrame</h3>

In [17]:
#do cleaning here
print(results)
venues_df = venues_df.drop(["location.distance", "location.city", "location.country", ""location.country"", "photos.count", "photos.groups"], axis = 1)
venues_df

{'meta': {'code': 403, 'errorType': 'rate_limit_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '60f4f96bab849b1373f4bb78'}, 'response': {}}


,id,name,categories,location.address,location.lat,location.lng,location.labeledLatLngs,location.cc,location.state,location.formattedAddress,venuePage.id,location.postalCode
0,4e8d9dcdd5fbbbb6b3003c7b,Brookbanks Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",Toronto,43.751976,-79.332140,"[{'label': 'display', 'lat': 43.75197604605557...",CA,ON,"[Toronto, Toronto ON, Canada]",600917367,NaN
0,58d5b8d0102f4722b70e487b,Careful & Reliable Painting,"[{'id': '5454144b498ec1f095bff2f2', 'name': 'C...",21 Valley Woods Rd,43.752622,-79.331957,"[{'label': 'display', 'lat': 43.752622, 'lng':...",CA,ON,"[21 Valley Woods Rd, Toronto ON M3A 0A1, Canada]",NaN,M3A 0A1
0,51d769f2498e3efc78bee6b9,649 Variety,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",NaN,43.754513,-79.331942,"[{'label': 'display', 'lat': 43.75451313339121...",CA,ON,"[Toronto ON, Canada]",NaN,NaN
0,5fc825cac5b8dd60907c56b3,Towns On The Ravine,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",35 Valley Woods Road,43.754754,-79.332552,"[{'label': 'display', 'lat': 43.7547545, 'lng'...",CA,ON,"[35 Valley Woods Road, Toronto ON M3A 2R5, Can...",1359110116,M3A 2R5
0,58f01a208f2c1a558cea5f7d,Sun Life,"[{'id': '5454144b498ec1f095bff2f2', 'name': 'C...",1211-44 Valley Woods Rd,43.754760,-79.332783,"[{'label': 'display', 'lat': 43.75476, 'lng': ...",CA,ON,"[1211-44 Valley Woods Rd, North York ON M3A 2R...",NaN,M3A 2R7
0,53e72d92498e336f61db5bf5,GTA Restoration,"[{'id': '52f2ab2ebcbc57f1066b8b3a', 'name': 'F...",18 Valley Woods Road #414,43.753396,-79.333477,"[{'label': 'display', 'lat': 43.75339615541568...",CA,ON,"[18 Valley Woods Road #414, Toronto ON M3A 0A1...",NaN,M3A 0A1
0,4cb11e2075ebb60cd1c4caad,Variety Store,"[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",29 Valley Woods Road,43.751974,-79.333114,"[{'label': 'display', 'lat': 43.75197441585782...",CA,ON,"[29 Valley Woods Road, Toronto ON, Canada]",NaN,NaN


<h3>Perform KMeans Clustering on Venues</h3>

In [19]:
kclusters = 5

coords = venues_df[["location.lat", "location.lng"]]

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(coords)
venues_df.insert(0, 'Cluster Labels', kmeans.labels_)


<h3>Visualizing Venues Clustering</h3>

In [22]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print((latitude, longitude))

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(venues_df['location.lat'], venues_df['location.lng'], venues_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

(43.6534817, -79.3839347)
